# [26 Punkte] Parsen eines Tic-Tac-Toe-Feldes

Tic-Tac-Toe ist ein beliebtes Strategiespiel. Auf einem quadratischen, drei mal drei Felder großen Spielfeld setzen zwei Spielerinnen abwechselnd ihr Zeichen in ein freies Feld. Eine Spielerin, die drei Zeichen in eine Zeile, Spalte, oder Diagonale gesetzt hat, gewinnt und beendet damit das Spiel. Werden alle Felder gefüllt, ohne dass dies einer Spielerin gelänge, endet das Spiel unentschieden. Sie wollen ein Programm schreiben, das Tic-Tac-Toe-Positionen aus Textdateien einliest und auf Gültigkeit prüft.

## [10 Punkte] Datei einlesen, prüfen, und konvertieren

### Hintergrund

Ein Spielzustand kann durch eine Liste von neun Zeichen aus einem Zeichensatz von drei Symbolen repräsentiert werden: Jedes der neun Felder ist mit dem Symbol der ersten Spielerin, dem Symbol der zweiten Spielerin, oder dem Symbol für einen leeren Platz gefüllt. Zusätzlich lassen sich die Symbole in einem Gitter von drei Zeilen und drei Spalten anordnen sowie optisch durch Separatoren trennen. In dieser Form lassen sich Spielstände in maschinen- und menschenlesbarer Form in Textdateien abspeichern.

### Grundlage

Man könnte als Symbol der Startspielerin das `X` wählen, als Symbol der zweiten Spielerin das `O`, für ein leeres Feld ein Leerzeichen, und die Zeilen und Spalten mit den Zeichen `-`, `|`, und `+` optisch trennen. Ein Beispiel für einen so visualisierten Spielzustand finden Sie in `undecided.tictactoe`:

In [ ]:
!cat undecided.tictactoe

### Aufgabenstellung

Schreiben Sie eine Funktion `parse_tictactoe`, die einen Spielzustand aus einer Datei einliest und parst:

```python
parse_tictactoe(filename, *, symbols=' XO', separators='|-+') -> tuple[int]
```

Der `filename` ist der Name der einzulesenden Datei. Die `symbols` sind dabei die Zeichen, die (in dieser Reihenfolge) ein leeres Feld, das Symbol der Startspielerin, und das Symbol der zweiten Spielerin repräsentieren. Die `separators` sind Separatoren (in dieser Reihenfolge) für Zeilen, Spalten, und Kreuzungspunkte.

Für einen gültigen Spielzustand soll die Funktion diesen als ein Tuple von genau 9 Integern codieren. Dabei soll `0` ein leeres Feld, `1` ein durch die Startspielerin besetztes Feld, und `-1` ein durch die zweite Spielerin besetztes Feld repräsentieren, zeilenweise in der Reihenfolge von links oben nach rechts unten. Für das oben angegebene Beispiel wäre die erwartete Rückgabe:

```python
>>> parse_tictactoe('example.tictactoe')
(1, 0, -1, -1, 1, 1, -1, 1, -1)
```

Da Dateien im Allgemeinen beliebigen Inhalt haben können, soll die Funktion prüfen, ob die Datei korrekt formatiert ist. Ist die Datei nicht korrekt formatiert, erzeugt (`raise`) die Funktion eine `Exception` Ihrer Wahl. Es gibt viele Möglichkeiten, wie eine Datei fehlerhaft formatiert sein kann. Sie müssen hier nur die folgenden Bedingungen prüfen:
   * die zweite und vierte Zeile enthält abwechselnd die letzten beiden `separators` (im Beispiel ist dies der String `'-+-+-'`; insbesondere dürfen diese Zeilen keine sonstigen Symbole enthalten)
   * Die erste Spielerin hat genau gleich viele ihrer Symbole oder genau ein Symbol mehr gesetzt zweite Spielerin (ansonsten wurden nicht abwechselnd, beginnend bei der ersten Spielerin, Symbole gesetzt) 

### Teilaufgaben

1. **[6 Punkte]** Die Funktion kodiert *korrekt formatierte* Dateien nach Vorgabe.
2. **[4 Punkte]** Die Funktion erzeugt für nach Vorgabe *fehlerhaft formatierte* Dateien eine `Exception`.

In [ ]:
# This is a code gap. Students can fill it.

[['X', '|', ' ', '|', 'O'],
 ['-', '+', '-', '+', '-'],
 ['O', '|', 'X', '|', 'X'],
 ['-', '+', '-', '+', '-'],
 ['O', '|', 'X', '|', 'O']]

Für die drei Beispiele können Sie Ihr Ergebnis hier prüfen:

In [ ]:
assert parse_tictactoe('undecided.tictactoe') == (1, 0, -1, -1, 1, 1, -1, 1, -1)
assert parse_tictactoe('first_player_wins.tictactoe') == (0, 0, 1, -1, -1, 1, 1, -1, 1)
assert parse_tictactoe('second_player_wins.tictactoe') == (-1, 0, -1, -1, 1, 1, -1, 1, 1)

Allerdings sollte Ihre Funktion Eingabedateien aller Art, also auch mit abweichenden Symbolen und Separatoren, verarbeiten können:

In [ ]:
import pathlib

import pytest
from pytest_nbgrader import loader

def test(task, subtask, custom=True):
    assert pytest.main(
        ['-qq', '-x',
         '-W', 'ignore::_pytest.warning_types.PytestAssertRewriteWarning',
         '--cases', pathlib.Path('tests') / task / f'{subtask}.yml'
        ] + custom * [f'tests/custom_test_tictactoe.py::Test{task}::test_{subtask}']
    ) is pytest.ExitCode.OK

loader.Submission.submit(parse_tictactoe)

In [ ]:
test('ParseTictactoe', 'valid_boards')

In [ ]:
test('ParseTictactoe', 'invalid_boards')

## [16 Punkte] Gewinnerin bestimmen

### Hintergrund

Eine Spielerin hat einen Spielzustand *gewonnen*, wenn sie alle Felder einer oder mehrerer Reihen, Spalten, oder Diagonalen des Spielfeldes mit ihren Symbolen gefüllt hat. Beispielsweise hat die erste Spielerin im Zustand `first_player_wins.tictactoe` gewonnen, während die zweite Spielerin im Zustand `second_player_wins` gewonnen hat:

In [ ]:
print('First player wins:')
!cat first_player_wins.tictactoe
print('\nSecond player wins:')
!cat second_player_wins.tictactoe

### Grundlage

Für die intermediäre Repräsentation nutzen Sie eine Dataclass namens `TicTacToe`:

In [ ]:
from libtictactoe import TicTacToe

Diese Klasse repräsentiert Zustände als `tuple[int]` der Länge neun wie im ersten Teil erzeugt. Die oben beschriebenen Zustände sind also:

In [ ]:
undecided = TicTacToe((1, 0, -1, -1, 1, 1, -1, 1, -1,))
first_player_won = TicTacToe((0, 0, 1, -1, -1, 1, 1, -1, 1))
second_player_won = TicTacToe((-1, 0, -1, -1, 1, 1, -1, 1, 1))

Mit dieser Implementierung können Sie auf die Zeilen, Spalten, und Diagonalen eines Spielzustands `state` über die Attribute `state.rows`, `state.cols`, und `state.diags` zugreifen und erhalten Informationen, welche Spielerin am Zug ist:

In [ ]:
print(
    f'{undecided.rows = } sind die drei Zeilen des Zustands (von oben nach unten),\n'
    f'{undecided.cols = } sind die drei Spalten (von links nach rechts), und\n'
    f'{undecided.diags =} sind die beiden Diagonalen des Zustandes.\n\n'
    f'{undecided.to_move % 3}. Spielerin ist am Zug.\n\n'
    f'{undecided}'
)

### Aufgabenstellung

Schreiben Sie eine Funktion `check_winner`, die für einen gegebenen Zustand des Spielfeldes ermittelt, wer gewonnen hat:

```python
check_winner(state: TicTacToe) -> int
```

Der `state` ist ein durch Instanzen der DataClass `TicTacToe` gegebener Spielzustand.

Für einen regelgerechten Spielzustand gibt die Funktion `1` zurück, wenn die erste Spielerin gewonnen hat, und `-1`, wenn die zweite Spielerin gewonnen hat. Wenn das Spiel (noch oder abschließend) unentschieden ist, wird eine `0` zurückgegeben:

```python
>>> check_winner(undecided), check_winner(first_player_won), check_winner(second_player_won)
(0, 1, -1)
```

Es können auch (nicht regelkonforme) Zustände auftreten, in denen beide Spielerinnen gewonnen haben, oder in denen eine Spielerin gewonnen hat, obwohl sie am Zug ist (und ihre Gegnerin trotzdem noch ein Symbol gesetzt hat). In diesem Fall soll eine `Exception` Ihrer Wahl auftreten.

### Teilaufgaben

1. **[10 Punkte]** Die Funktion gibt für *regelgerechte* Zustände die Siegerin nach Vorgabe aus.
2. **[6 Punkte]** Die Funktion erzeugt für *nicht regelgerechte* Zustände eine `Exception`.\
   *Hinweis*: Es genügt zu prüfen, dass die Spielerin, die am Zug ist (`state.to_move`), nicht gewonnen hat.

In [ ]:
# This is a code gap. Students can fill it.

Mit drei gültigen Beispielzuständen können Sie Ihre Antwort hier manuell testen:

In [ ]:
assert check_winner(first_player_won) == 1
assert check_winner(second_player_won) == -1
assert check_winner(undecided) == 0

Nur für ungültige Beispielzustände sollten Sie eine Exception erhalten (dann läuft die nächste Zelle fehlerfrei durch):

In [ ]:
check_winner(first_player_won)
check_winner(second_player_won)
check_winner(undecided)

both_players_won_invalid = TicTacToe((-1, -1, -1, 1, 1, 1, 0, 0, 0))
player_to_move_won_invalid = TicTacToe((1, 1, 1, -1, 0, -1, -1, 0, 0))

with pytest.raises(Exception):
    check_winner(both_players_won_invalid)

with pytest.raises(Exception):
    check_winner(player_to_move_won_invalid)

Allerdings funktioniert Ihr Code für andere Fälle auch:

In [ ]:
import pathlib

import pytest
from pytest_nbgrader import loader

def test(task, subtask, custom=True):
    assert pytest.main(
        ['-qq', '-x',
         '-W', 'ignore::_pytest.warning_types.PytestAssertRewriteWarning',
         '--cases', pathlib.Path('tests') / task / f'{subtask}.yml'
        ] + custom * [f'tests/custom_test_tictactoe.py::Test{task}::test_{subtask}']
    ) is pytest.ExitCode.OK
    
loader.Submission.submit(check_winner)

In [ ]:
test('CheckWinner', 'valid_states', custom=False)

In [ ]:
test('CheckWinner', 'invalid_states', custom=False)